In [1]:
import pandas as pd
from fastai.metrics import *
from fastai.tabular import *

In [2]:
train = pd.read_csv('../data/engineered/powerelectricity_train.csv')
test = pd.read_csv('../data/engineered/powerelectricity_test.csv')

In [3]:
len(train), len(test)

(54039, 6719)

In [4]:
all_data = pd.concat([train, test])
all_data.drop(['rms_current', 'electric_power'],axis='columns', inplace=True)

In [5]:
dep_var = 'delta_y'
cat_names = all_data.select_dtypes(exclude=np.number).columns.tolist()
cont_names = all_data.select_dtypes(include=np.number).columns.tolist()
cont_names.remove('delta_y')
procs=[FillMissing, Categorify, Normalize]

In [6]:
test = TabularList.from_df(all_data.iloc[len(train):-1].copy(), cat_names=cat_names, cont_names=cont_names)

In [7]:
data = (TabularList.from_df(all_data, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(len(train),len(all_data))))
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())

In [8]:
learn = tabular_learner(data, layers=[200,100], metrics=rmse)

In [9]:
learn.fit(5, 1e-2)

epoch,train_loss,valid_loss,root_mean_squared_error,time
0,1.605323,2.924095,1.327674,00:13
1,1.299297,2.598676,1.283161,00:10
2,1.090641,2.246449,1.191468,00:11
3,1.113791,2.246405,1.227011,00:13
4,1.022835,2.629779,1.244033,00:12
